# Normalization

In [25]:
import pandas as pd
import numpy as np
import re
import emoji
import string
import contractions
from functools import reduce

import nltk
from nltk.corpus import stopwords, words, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize

# Da inserire nelle dipendenze
from spellchecker import SpellChecker

In [26]:
#Define stop words for text cleaning
stop_words = set(stopwords.words('english'))

stop_words.add("mkr")

lemmatizer = WordNetLemmatizer()

ps = PorterStemmer()


In [27]:
#Replace all textual emojis in the string with null character
def remove_textual_emojis(tweet): 
    tweet = re.sub(r"(:[DPp3Oo\(\)])|(:'[\)\(])|(;[\)\(])|(-.-)|(^[._]^)|(x[\(\)])", '', tweet)
    return tweet #emoji.replace_emoji(tweet,'')


#Replace links and mentions (@) with null character
def remove_links_mentions(tweet): 
    return re.sub(r"((?:\@|https?\:\/\/|www)\S+)|(^RT)", '', tweet)


#Remove all hashtags at the end of the sentence and remove the # symbol from all others
def remove_hashtag(tweet):
    tweet = re.sub(r"(\s+#[\w-]+)+\s*$", '', tweet)
    return re.sub(r"#([\w-]+)", r'\1', tweet)


#Remove multiple spaces (2+) and remove spaces at the beginning and end of the sentence 
def remove_spaces(tweet):
    tweet = re.sub(r"\s{2,}", ' ', tweet)
    tweet = re.sub(r"^\s", '', tweet)
    return re.sub(r"\s$", '', tweet)


#Remove not ASCII characters (it includes not-textual emojis)
def remove_not_ASCII(tweet):
    return tweet.encode("ascii", errors="ignore").decode()


#Expand contractions (e.g.: can't => cannot)
def remove_contractions(tweet):
    return contractions.fix(tweet,slang=True)


#Remove all stopwords
def remove_stopwords(tweet):
    word_tokens = word_tokenize(tweet)
    
    # checks whether they are present in stop_words or not
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
            
    return ' '.join(filtered_sentence)


#Remove punctuation symbols
def remove_punctuation(tweet):
    return remove_spaces(tweet.translate(str.maketrans("", "", string.punctuation)))


#Stemming
def stemming(tweet):
    tmp_tweet = word_tokenize(tweet)
    return reduce(lambda x, y: x + ps.stem(y), tmp_tweet, "")


#Remove multiple doubles in each word of the tweet (e.g.: coooll => cooll)
def remove_elongated_words(tweet):
    return re.sub(r'(.)\1+', r'\1\1', tweet)


#Remove & and $ symbols
def remove_special_characters(tweet):
    return re.sub(r'[&$]|amp', '',tweet)


#Remove short tweets
def remove_short_tweets(tweet, min_words=3):
    words = tweet.split()
    return tweet if len(words) >= min_words else ""


In [28]:
# Check words spelling

spell = SpellChecker()

def correct_spelling(tweet):
    words = word_tokenize(tweet)
    misspelled = spell.unknown(words)
    corrected_words = []

    for word in words:
        if word in misspelled:
            correction = spell.correction(word)
            corrected_words.append(correction if correction is not None else word)
        corrected_words.append(word)
    return ' '.join(corrected_words)


# Abbreviations and non-english words

abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

nltk.download('words')
nltk.download('wordnet')

# Combine wordnet and the standard list of words in NLTK
english_words = set(words.words())
english_words.update(w for w in wordnet.words())

# Add abbreviations
english_words.update(abbreviations)

def remove_non_english(tweet):
    words = word_tokenize(tweet)
    filtered_words = []
    
    for word in words:
        if word in english_words:
            filtered_words.append(word)

    return ' '.join(filtered_words)

[nltk_data] Downloading package words to
[nltk_data]     /Users/chiarapiccolo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chiarapiccolo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
def lemmatization_with_pos(sentence):
    wnl = WordNetLemmatizer()
    tagged_words = pos_tag(word_tokenize(sentence))
    lemmatized_words = []
    
    for word, tag in tagged_words:
        wntag = tag[0].lower()  # Get the first character of the tag
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None  # Map POS tags to WordNet POS tags
        if not wntag:
            lemma = word  # If the POS tag is not recognized, use the word as is
        else:
            lemma = wnl.lemmatize(word, pos=wntag)  # Lemmatize the word with its POS tag
        lemmatized_words.append(lemma)
    
    return ' '.join(lemmatized_words)

In [30]:
def clean_normalized_df(df):
    #Delete empty tweets after the normalization
    df=df.drop(df[(df.tweet_text == r'')].index)  # Rimosse 273 entries
    
    #Check duplicates after the normalization
    df = df.drop_duplicates()

    return df

For each model, we need apply different normalization techniques.

In [20]:
def Bayes_normalization(tweet):
    tweet = remove_links_mentions(tweet)
    tweet = tweet.lower()
    tweet = remove_hashtag(tweet)
    tweet = remove_special_characters(tweet)
 
    tweet = remove_spaces(tweet)
    tweet = remove_textual_emojis(tweet)
    tweet = remove_not_ASCII(tweet)
    tweet = remove_contractions(tweet)
    tweet = remove_stopwords(tweet)
    tweet = remove_punctuation(tweet)
    tweet = remove_elongated_words(tweet)
    
    tweet = lemmatization_with_pos(tweet)
    
    tweet = remove_short_tweets(tweet, min_words=3)
    return tweet

def Bayes_preprocessing(df):
    df['tweet_text'] = df['tweet_text'].apply(Bayes_normalization)
    df = clean_normalized_df(df)
    return df

In [21]:
def Transformers_normalization(tweet):
    tweet = remove_links_mentions(tweet)
    tweet = tweet.lower()
    tweet = remove_hashtag(tweet)
    tweet = remove_special_characters(tweet)
 
    tweet = remove_spaces(tweet)
    tweet = remove_textual_emojis(tweet)
    tweet = remove_not_ASCII(tweet)
    return tweet

def Transformers_preprocessing(df):
    df['tweet_text'] = df['tweet_text'].apply(Transformers_normalization)
    df = clean_normalized_df(df)
    return df

In [31]:
def LSTM_normalization(tweet):
    tweet = remove_links_mentions(tweet)
    tweet = tweet.lower()
    tweet = remove_hashtag(tweet)
    tweet = remove_special_characters(tweet)
 
    tweet = remove_spaces(tweet)
    tweet = remove_textual_emojis(tweet)
    tweet = remove_not_ASCII(tweet)
    tweet = remove_contractions(tweet)
    tweet = remove_stopwords(tweet)
    tweet = remove_punctuation(tweet)
    tweet = remove_elongated_words(tweet)
    
    tweet = correct_spelling(tweet)
    tweet = remove_non_english(tweet)
    tweet = remove_short_tweets(tweet, min_words=3)

    return tweet

def LSTM_preprocessing(df):
    df['tweet_text'] = df['tweet_text'].apply(LSTM_normalization)
    df = clean_normalized_df(df)
    return df

In [32]:
pool = {"../../data/train_tweets.csv", "../../data/eval_tweets.csv", "../../data/test_tweets.csv"}


In [ ]:
for file in pool:
    df = pd.read_csv(file)
    df = Bayes_preprocessing(df)
    file = file.split(".csv")[0]
    df.to_csv(file+"_Naive_Bayes.csv", index=False)

In [33]:
for file in pool:
    df = pd.read_csv(file)
    df = LSTM_preprocessing(df)
    file = file.split(".csv")[0]
    df.to_csv(file+"_LSTM.csv", index=False)

In [ ]:
for file in pool:
    df = pd.read_csv(file)
    df = Transformers_preprocessing(df)
    file = file.split(".csv")[0]
    df.to_csv(file+"_Transformers.csv", index=False)